In [24]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "majisouvik1099"
os.environ['KAGGLE_KEY'] = "9464225218d56f8bfea9dd9cc437489f"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 94% 106M/112M [00:00<00:00, 127MB/s] 
100% 112M/112M [00:00<00:00, 121MB/s]


In [6]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


# def compute_hog(img):
#     resized_img = resize(img, (128*4, 64*4))
#     fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
#                     cells_per_block=(2, 2), visualize=True, multichannel=True)
#     return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

# def lbp_calculated_pixel(img, x, y):
#     center = img[x][y]
#     val_ar = []
#     val_ar.append(get_pixel(img, center, x-1, y+1))
#     val_ar.append(get_pixel(img, center, x, y+1))
#     val_ar.append(get_pixel(img, center, x+1, y+1))
#     val_ar.append(get_pixel(img, center, x+1, y))
#     val_ar.append(get_pixel(img, center, x+1, y-1))
#     val_ar.append(get_pixel(img, center, x, y-1))
#     val_ar.append(get_pixel(img, center, x-1, y-1))
#     val_ar.append(get_pixel(img, center, x-1, y))

#     power_val = [1, 2, 4, 8, 16, 32, 64, 128]
#     val = 0
#     for i in range(len(val_ar)):
#         val += val_ar[i] * power_val[i]
#     return val

# def calcLBP(img):
#     height, width, channel = img.shape
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_lbp = np.zeros((height, width,3), np.uint8)
#     for i in range(0, height):
#         for j in range(0, width):
#              img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
#     hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
#     return hist_lbp.flatten()


In [7]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 116MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [8]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                # hog_feature = compute_hog(image)
                # lbp_feature = calcLBP(image)
                cnn_feature = extract_features(image_path, resnet).numpy()
                # hog_feature = hog_feature.reshape(-1)
                # lbp_feature = lbp_feature.reshape(-1)
                cnn_feature = cnn_feature.flatten()
                # combined_feature = np.concatenate((hog_feature, lbp_feature, cnn_feature))

                X.append(cnn_feature)
                y.append(folder_name)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [10]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

accuracy_log_reg = 0
accuracy_log_reg_old = -1
epochs = 10
while ((accuracy_log_reg - accuracy_log_reg_old) > 0.01):
  accuracy_log_reg_old = accuracy_log_reg
  logistic_model = LogisticRegression(max_iter = epochs, multi_class='ovr')
  logistic_model.fit(X_train, y_train_encoded)
  y_pred_log_reg = logistic_model.predict(X_test)
  accuracy_log_reg = accuracy_score(y_test_encoded, y_pred_log_reg)
  epochs+= 100


print("Accuracy on test data:", accuracy_log_reg)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_log_reg, target_names=label_encoder.classes_))


Accuracy on test data: 0.7984496124031008
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.93      0.81      0.87        16
     Colin_Powell       0.73      0.92      0.81        38
  Donald_Rumsfeld       0.81      0.65      0.72        26
    George_W_Bush       0.83      0.92      0.87       109
Gerhard_Schroeder       0.79      0.63      0.70        30
      Hugo_Chavez       0.89      0.47      0.62        17
       Tony_Blair       0.64      0.64      0.64        22

         accuracy                           0.80       258
        macro avg       0.80      0.72      0.75       258
     weighted avg       0.80      0.80      0.79       258



In [27]:
y_pred_log_reg = logistic_model.predict(X_train)
accuracy_log_reg = accuracy_score(y_train_encoded, y_pred_log_reg)
print("Logistic Regression")
print("Accuracy on train set:", accuracy_log_reg)

Logistic Regression
Accuracy on train set: 1.0


In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

logistic_model.fit(X_train_pca, y_train_encoded)
y_pred_log_reg = logistic_model.predict(X_test_pca)
accuracy = accuracy_score(y_test_encoded, y_pred_log_reg)
print("PCA + Logistic Regression Classifier")
print("Accuracy on test set:", accuracy)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_log_reg, target_names=label_encoder.classes_))

PCA + Logistic Regression Classifier
Accuracy on test set: 0.7441860465116279
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.86      0.75      0.80        16
     Colin_Powell       0.67      0.87      0.76        38
  Donald_Rumsfeld       0.65      0.58      0.61        26
    George_W_Bush       0.82      0.85      0.83       109
Gerhard_Schroeder       0.70      0.53      0.60        30
      Hugo_Chavez       0.75      0.53      0.62        17
       Tony_Blair       0.61      0.64      0.62        22

         accuracy                           0.74       258
        macro avg       0.72      0.68      0.69       258
     weighted avg       0.74      0.74      0.74       258



In [31]:
y_pred_log_reg = logistic_model.predict(X_train_pca)
accuracy_log_reg = accuracy_score(y_train_encoded, y_pred_log_reg)
print("PCA + Logistic Regression Classifier")
print("Accuracy on train set:", accuracy_log_reg)

ValueError: X has 202 features, but LogisticRegression is expecting 6 features as input.

In [29]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)
X_train_lda = lda.fit_transform(X_train, y_train_encoded)
X_test_lda = lda.transform(X_test)
logistic_model.fit(X_train_lda, y_train_encoded)
y_pred_log_reg = logistic_model.predict(X_test_lda)
accuracy_gnb = accuracy_score(y_test_encoded, y_pred_log_reg)
print("LDA + Logistic Regression")
print("Accuracy on Test set after LDA:", accuracy_log_reg)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_log_reg, target_names=label_encoder.classes_))

LDA + Logistic Regression
Accuracy on Test set after LDA: 0.7984496124031008
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.85      0.69      0.76        16
     Colin_Powell       0.78      0.92      0.84        38
  Donald_Rumsfeld       0.85      0.88      0.87        26
    George_W_Bush       0.91      0.87      0.89       109
Gerhard_Schroeder       0.81      0.73      0.77        30
      Hugo_Chavez       0.86      0.71      0.77        17
       Tony_Blair       0.64      0.82      0.72        22

         accuracy                           0.84       258
        macro avg       0.81      0.80      0.80       258
     weighted avg       0.84      0.84      0.84       258



In [30]:
y_pred_log_reg = logistic_model.predict(X_train_lda)
accuracy_log_reg = accuracy_score(y_train_encoded, y_pred_log_reg)
print("LDA + Logistic Regression")
print("Accuracy on Train set:", accuracy_log_reg)

LDA + Logistic Regression
Accuracy on Train set: 0.996116504854369
